In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd

def global_elec_mix(file_path, run_name,output_folder, called = False):
    if called == True:
        var_Fout = pd.read_csv(file_path + 'VAR_FOut_' + run_name + '.csv', sep = ',')

        elec_prod = var_Fout[var_Fout['1'] == 'ELC']
        elec_prod['prod_type'] = elec_prod['2'].str.split('_').str[1]
        elec_prod['group_type'] = elec_prod['prod_type'].astype(str).str.cat(elec_prod['3'].astype(str), sep='_')
        elec_prod['group_prod_type'] = elec_prod.groupby('group_type')['8'].transform('sum')
        elec_prod['group_prod_type'] = (elec_prod['group_prod_type']*277)/1000
        elec_prod = elec_prod[~elec_prod['group_type'].duplicated()]

        values1 = np.linspace(0, 1, 20)
        values2 = np.linspace(0, 1, 20)
        values3 = np.linspace(0, 1, 16)
        # Create two colormaps
        cmap1 = plt.cm.tab20
        cmap2 = plt.cm.tab20b
        cmap3 = plt.cm.Set3


        colors1 = cmap1(values1)
        colors2 = cmap2(values2)
        colors3 = cmap3(values3)


        # Combine the colors from both colormaps
        combined_colors = np.vstack((colors1, colors2,colors3))

        # Create a new colormap using the combined colors
        combined_cmap = ListedColormap(combined_colors)

        pivot_df = elec_prod.pivot(index='3', columns='prod_type', values='group_prod_type')
        column_sums = pivot_df.sum(skipna=True)

        # Step 2: Sort the columns based on their sums
        sorted_columns = column_sums.sort_values(ascending=False)

        # Step 3: Reindex the DataFrame with the sorted column order
        sorted_df = pivot_df.reindex(sorted_columns.index, axis=1)
        sorted_df.head()
        # Plot with the discrete color map
        percentages = sorted_df.div(sorted_df.sum(axis=1), axis=0) * 100

        # Create a figure with two subplots
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        # Plot the original data in the first subplot
        sorted_df.plot(kind='bar', stacked=True, colormap=combined_cmap, ax=axs[0], legend= False)
        axs[0].set_xlabel('Year')
        axs[0].set_ylabel('TWh')
        axs[0].set_title('World Electricity Generation Mix')

        # Plot the percentages in the second subplot
        percentages.plot(kind='bar', stacked=True, colormap=combined_cmap, ax=axs[1], legend=False)
        axs[1].set_xlabel('Year')
        axs[1].set_ylabel('Percentage')
        axs[1].set_title('World Electricity Generation Mix (100%)')

        # Create a single legend for both subplots
        handles, labels = axs[0].get_legend_handles_labels()
        fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.35), ncol=7)

        # Adjust layout
        plt.tight_layout()

        plt.savefig(output_folder + 'world_elec_mix.pdf'
                    , format ='pdf'
                    ,  bbox_inches='tight')
    else:
        pass